In [1]:
import cv2

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while cv2.waitKey(33) < 0:
    ret, frame = capture.read()
    cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()

In [ ]:
영상에 mediapipe 적용. 

In [3]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_detection = mp.solutions.face_detection

#파일 위치 미리 지정
input_video_path = "S#17 촬영본.mp4"
save_video_path = 'S#17 촬영본_output.mp4'

cap = cv2.VideoCapture(input_video_path)

#재생할 파일의 넓이와 높이
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
#video controller
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter(save_video_path, fourcc, 30.0, (int(width), int(height)))


with mp_face_detection.FaceDetection(model_selection=1, 
        min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("카메라를 찾을 수 없습니다.")
            # 웹캠을 불러올 경우는 'continue', 동영상을 불러올 경우 'break'를 사용합니다.
            break

        # 필요에 따라 성능 향상을 위해 이미지 작성을 불가능함으로 기본 설정합니다.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # 포즈 주석을 이미지 위에 그립니다.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
            
        # 보기 편하게 이미지를 좌우 반전합니다.
        cv2.imshow('MediaPipe Face Detection', image) #코랩에서 돌릴거면 imshow()문은 주석처리할 것.
        out.write(image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
out.release()
cv2.destroyAllWindows()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


카메라를 찾을 수 없습니다.


In [4]:
import cv2
import mediapipe as mp
import numpy as np

# Mediapipe의 Face 모듈을 사용하기 위한 초기화
mp_face = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face.FaceDetection()

# 원본 영상 파일 경로
video_path = 'S#17 촬영본.mp4'

# 원본 영상 파일에서 비디오 캡쳐를 초기화합니다.
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    # 프레임을 읽어옵니다.
    ret, frame = cap.read()
    if not ret:
        print("비디오 읽기 실패")
        break

    # BGR을 RGB로 변환합니다.
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Face Detection을 적용합니다.
    result = face_detection.process(rgb_frame)

    # Face Detection 결과가 있을 경우
    if result.detections:
        for detection in result.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            # Face landmark를 가져옵니다.
            landmarks = detection.location_data.relative_keypoints
            lip_left_x = int(landmarks[mp_face.FaceKeyPoint.LOWER_LIP_LEFT].x * iw)
            lip_left_y = int(landmarks[mp_face.FaceKeyPoint.LOWER_LIP_LEFT].y * ih)

            lip_right_x = int(landmarks[mp_face.FaceKeyPoint.LOWER_LIP_RIGHT].x * iw)
            lip_right_y = int(landmarks[mp_face.FaceKeyPoint.LOWER_LIP_RIGHT].y * ih)

            # 입술 좌표를 출력합니다.
            print("입술 왼쪽 좌표 (x, y):", lip_left_x, lip_left_y)
            print("입술 오른쪽 좌표 (x, y):", lip_right_x, lip_right_y)

            # 입술 좌표를 표시합니다.
            cv2.circle(frame, (lip_left_x, lip_left_y), 5, (0, 255, 0), -1)
            cv2.circle(frame, (lip_right_x, lip_right_y), 5, (0, 255, 0), -1)

    # 화면에 출력합니다.
    cv2.imshow('Mediapipe Face', frame)

    # 'q' 키를 누르면 종료합니다.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 사용한 자원을 해제합니다.
cap.release()
cv2.destroyAllWindows()


AttributeError: LOWER_LIP_LEFT

In [6]:
pip install mediapipe --upgrade

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp

# Mediapipe Face Mesh 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# 동영상 파일 경로
video_path = 'S#17 촬영본.mp4'  # 동영상 파일 경로를 적절히 수정하세요.

# OpenCV 초기화
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Face Mesh를 이용하여 얼굴 landmark 검출
    results = face_mesh.process(rgb_frame)

    # 얼굴 landmark가 검출되었다면
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 입술 좌표 얻기
            for idx, landmark in enumerate(face_landmarks.landmark):
                height, width, _ = frame.shape
                cx, cy = int(landmark.x * width), int(landmark.y * height)

                # 입술 부분만 출력
                if 48 <= idx <= 59:
                    cv2.circle(frame, (cx, cy), 2, (0, 255, 0), -1)

    # 결과를 화면에 출력
    cv2.imshow('Face Mesh', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # 'Esc' 키를 누르면 종료
        break

# 정리 작업
cap.release()
cv2.destroyAllWindows()


2023-12-08 11:40:10.642407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
